In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import re
import sklearn
nltk.download("stopwords")
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajayb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ajayb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajayb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
true = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")
true["result"]=1
fake["result"]=0
print(true.head(5))
print(fake.head(5))


                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  result  
0  December 31, 2017        1  
1  December 29, 2017        1  
2  December 31, 2017        1  
3  December 30, 2017        1  
4  December 29, 2017        1  
                                               tit

In [8]:
true.shape,fake.shape

((21417, 5), (23481, 5))

In [9]:
df = pd.concat([true,fake],axis =0)
df.head(5)

,title,text,subject,date,result
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [10]:
print(df["result"].value_counts())

print(df.columns)

df.shape

result
0    23481
1    21417
Name: count, dtype: int64
Index(['title', 'text', 'subject', 'date', 'result'], dtype='object')


(44898, 5)

In [11]:
df = df.drop(["title","subject","date"],axis=1)
df.head(2)

,text,result
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1


In [12]:
stop_words = set(stopwords.words("english"))
lem = WordNetLemmatizer()

def clear(text):
    text = BeautifulSoup(text,"html.parser").get_text()
    text = contractions.fix(text)
    text = re.sub(r'[^A-Z a-z\s]'," ", text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if len(t)>1]
    tokens = [lem.lemmatize(t) for t in tokens if t not in stop_words]
    return " ".join(tokens)



In [13]:
df["text"]= df["text"].apply(clear)
df["text"].head(5)

C:\Users\ajayb\AppData\Local\Temp\ipykernel_29160\3946799348.py:5: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text,"html.parser").get_text()


0    washington reuters head conservative republica...
1    washington reuters transgender people allowed ...
2    washington reuters special counsel investigati...
3    washington reuters trump campaign adviser geor...
4    seattle washington reuters president donald tr...
Name: text, dtype: object

In [15]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['text'])
y = df['result']


In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y ,test_size = 0.2, random_state= 42 ,stratify = y)
x_train.shape,y_train.shape

((35918, 106890), (35918,))

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression()
model.fit(x_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [18]:
y_pred = model.predict(x_test)
print("accuracy_score=",accuracy_score(y_test,y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

accuracy_score= 0.9858574610244989

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4696
           1       0.98      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4616   80]
 [  47 4237]]


In [19]:
news = "a recent paper about financial fake‑news detection, useful if you want to test your model on finance‑related news"
toke= clear(news)
news_vector = tfidf.transform([toke])
predict = model.predict(news_vector)
prob = model.predict_proba(news_vector)
p=prob[0][0]
if (p) >= 0.5:
        print(f"Real news! (confidence: {p:.2f})")
else:
        print(f"fake news. (confidence: {1-p:.2f})")

Real news! (confidence: 0.90)


In [20]:
# model 2 
from gensim.models import Word2Vec
sentences = [clear(W).split() for W in df["text"]]
dimension = 100
W2v = Word2Vec(   sentences,
                  vector_size = dimension,
                  window = 5,
                  min_count = 2,
                  sg = 1)

In [21]:
def vecsent(tokens):
    vectors=[]
    for word in tokens:
        if word in W2v.wv:
            vectors.append(W2v.wv[word])
    if len(vectors) == 0:
        return np.zeros(dimension)    
    else:
        return np.mean(vectors,axis = 0).astype(np.float32)

xw = np.vstack([vecsent(toks.split()) for toks in df["text"]])
yw = df["result"].values


In [22]:
from sklearn.model_selection import train_test_split
Xw_train, Xw_test , yw_train , yw_test = train_test_split(xw,yw,random_state=42, test_size=0.2,stratify= yw)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

x_train_t = torch.tensor(Xw_train,dtype=torch.float32).to(device)
y_train_t = torch.tensor(yw_train, dtype=torch.float32).view(-1,1).to(device)
x_test_t = torch.tensor(Xw_test,dtype=torch.float32).to(device)
y_test_t = torch.tensor(yw_test,dtype=torch.float32).view(-1,1).to(device)


In [32]:
#creating model 
modelw2v = nn.Sequential(nn.Linear(dimension,75),
                         nn.Linear(75,75),
                         nn.ReLU(),
                         nn.Linear(75,50),
                         nn.ReLU(),
                         nn.Linear(50,25),
                         nn.ReLU(),
                         nn.Linear(25,10),
                         nn.ReLU(),
                         nn.Linear(10,5),
                         nn.ReLU(),
                         nn.Linear(5,1)
).to(device)

In [33]:
criterion = nn.BCEWithLogitsLoss()
optimizer = (optim.Adam(modelw2v.parameters(), lr = 0.001))

In [34]:
# training model 
epochs = 800
batch_size = 20

for epoch in range(epochs):
    modelw2v.train()
    epoch_loss = 0.0

    for i in range(0,len(x_train_t),batch_size):
        x_batch = x_train_t[i:i+batch_size]
        y_batch = y_train_t[i:i+batch_size]
        
        logits = modelw2v(x_batch)
        lossE = criterion(logits,y_batch)

        #backward propagation
        optimizer.zero_grad()
        lossE.backward()
        optimizer.step()

        epoch_loss += lossE.item() 

    avg_loss = epoch_loss/len(x_train_t)
    print(f"Epoch {epoch+1} , Loss: {avg_loss:.4f}")








Epoch 1 , Loss: 0.0096
Epoch 2 , Loss: 0.0070
Epoch 3 , Loss: 0.0065
Epoch 4 , Loss: 0.0063
Epoch 5 , Loss: 0.0061
Epoch 6 , Loss: 0.0059
Epoch 7 , Loss: 0.0057
Epoch 8 , Loss: 0.0054
Epoch 9 , Loss: 0.0052
Epoch 10 , Loss: 0.0050
Epoch 11 , Loss: 0.0049
Epoch 12 , Loss: 0.0047
Epoch 13 , Loss: 0.0046
Epoch 14 , Loss: 0.0045
Epoch 15 , Loss: 0.0044
Epoch 16 , Loss: 0.0044
Epoch 17 , Loss: 0.0042
Epoch 18 , Loss: 0.0042
Epoch 19 , Loss: 0.0041
Epoch 20 , Loss: 0.0040
Epoch 21 , Loss: 0.0039
Epoch 22 , Loss: 0.0039
Epoch 23 , Loss: 0.0038
Epoch 24 , Loss: 0.0037
Epoch 25 , Loss: 0.0036
Epoch 26 , Loss: 0.0036
Epoch 27 , Loss: 0.0036
Epoch 28 , Loss: 0.0035
Epoch 29 , Loss: 0.0035
Epoch 30 , Loss: 0.0034
Epoch 31 , Loss: 0.0034
Epoch 32 , Loss: 0.0034
Epoch 33 , Loss: 0.0032
Epoch 34 , Loss: 0.0032
Epoch 35 , Loss: 0.0031
Epoch 36 , Loss: 0.0031
Epoch 37 , Loss: 0.0029
Epoch 38 , Loss: 0.0031
Epoch 39 , Loss: 0.0030
Epoch 40 , Loss: 0.0029
Epoch 41 , Loss: 0.0028
Epoch 42 , Loss: 0.0028
E

In [39]:
modelw2v.eval()
with torch.no_grad():
    logit = modelw2v(x_test_t)
    prob = torch.sigmoid(logit)
    pred_lab = (prob>=0.5).float()


pred_np = pred_lab.cpu().numpy()
true_np = y_test_t.cpu().numpy()

acc = accuracy_score(true_np, pred_np)
print(f'accuracy of w2v and ann : {acc:.4f}')






accuracy of w2v and ann : 0.9780
